In [1]:
# 这里主要介绍一下mysql
import pymysql

# 最简单的例子
db = pymysql.connect(host='192.168.1.50',user='root', password='xiaoyou', port=30000)
cursor = db.cursor()
# 这里我们查看一下数据库的版本
cursor.execute('SELECT VERSION()')
data = cursor.fetchone()
print('Database version:', data)
# 这里我们新增一个数据库
cursor.execute("CREATE DATABASE spiders DEFAULT CHARACTER SET utf8")
db.close()

Database version: ('8.0.27',)


In [4]:
# 下面我们新建一个数据表，这里需要使用db参数来指定数据库
db = pymysql.connect(host='192.168.1.50',user='root', password='xiaoyou', port=30000, db='spiders')
cursor = db.cursor()
# 这里新建一个student的表
sql = 'CREATE TABLE IF NOT EXISTS students (id VARCHAR(255) NOT NULL, name VARCHAR(255) NOT NULL, age INT NOT NULL, PRIMARY KEY (id))'
cursor.execute(sql)

0

In [7]:
# 新建好了表后，我们就可以插入一条数据了
sql = 'INSERT INTO students(id, name, age) values(%s, %s, %s)'

id = '20120001'
user = 'Bob'
age = 20

try:
    # 这里执行我们的sql语句来插入数据。这里我们使用了事务，需要执行commit
    cursor.execute(sql, (id, user, age))
    db.commit()
except:
    db.rollback()

In [8]:
# 这里我们可以使用格式化来读取数据
data = {
    'id': '20120001',
    'name': 'Bob',
    'age': 20
}
table = 'students'
keys = ', '.join(data.keys())
values = ', '.join(['%s'] * len(data))
# 使用这种方式插入数据就非常灵活了
sql = 'INSERT INTO {table}({keys}) VALUES ({values})'.format(table=table, keys=keys, values=values)
print(sql)

INSERT INTO students(id, name, age) VALUES (%s, %s, %s)


In [9]:
# 下面我们试一下更新数据
# 修改bob的年龄为25
sql = 'UPDATE students SET age = %s WHERE name = %s'
try:
    cursor.execute(sql, (25, 'Bob'))
    db.commit()
except:
    db.rollback()
db.close()

In [12]:
# 如果出现重复数据时要怎么操作
data = {
    'id': '20120001',
    'name': 'Bob',
    'age': 21
}

table = 'students'
keys = ', '.join(data.keys())
values = ', '.join(['%s'] * len(data))

db = pymysql.connect(host='192.168.1.50',user='root', password='xiaoyou', port=30000, db='spiders')
cursor = db.cursor()
# 这里可以使用ON DUPLICATE KEY UPDATE，就是当主键存在的时候执行更新操作，否则执行插入操作
sql = 'INSERT INTO {table}({keys}) VALUES ({values}) ON DUPLICATE KEY UPDATE'.format(table=table, keys=keys,
                                                                                     values=values)
update = ','.join([" {key} = %s".format(key=key) for key in data])
sql += update
print(sql)
try:
    if cursor.execute(sql, tuple(data.values()) * 2):
        print('Successful')
        db.commit()
except:
    print('Failed')
    db.rollback()
db.close()

INSERT INTO students(id, name, age) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE id = %s, name = %s, age = %s
Successful


In [ ]:
# 上面那个例子因为数据已经存在了，所以会直接执行更新操作，更新年龄为21

In [14]:
# 下面演示一下删除数据
table = 'students'
condition = 'age > 20'

db = pymysql.connect(host='192.168.1.50',user='root', password='xiaoyou', port=30000, db='spiders')
cursor = db.cursor()
sql = 'DELETE FROM  {table} WHERE {condition}'.format(table=table, condition=condition)
try:
    cursor.execute(sql)
    db.commit()
except:
    db.rollback()

db.close()

In [15]:
# 最后就是查询操作
sql = 'SELECT * FROM students WHERE age >= 20'
db = pymysql.connect(host='192.168.1.50',user='root', password='xiaoyou', port=30000, db='spiders')
cursor = db.cursor()
try:
    cursor.execute(sql)
    print('Count:', cursor.rowcount)
    # 这里我们这里我们使用游标不断遍历数据
    row = cursor.fetchone()
    while row:
        print('Row:', row)
        row = cursor.fetchone()
except:
    print('Error')

Count: 0
